In this notebook we evaluate performance of pretrained RoBERTa model.

# PREREQUISITES:

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score
import os
from huggingface_hub import login

In [ ]:
login() # login to your hugging face account to use model

In [2]:
current_directory = os.getcwd() # get current directory
file_path = os.path.join(current_directory, '..', "..", 'data', 'processed', 'clean_data.csv') # navigate to folder with preprocessed data
data = pd.read_csv(file_path)

In [3]:
print(data.head(5))

                                        heading_text  bias_rating
0  chicago gun violence spikes and increasingly f...            0
1  'bullets just came from nowhere': fourth of ju...            1
2  dozens of shootings across us mark bloody july...            2
3  federal government will run out of cash on oct...            2
4  yellen tells congress that u.s. will run out o...            0


# Data splitting

In [16]:
# devide data into text and labels
X = data['heading_text'].values
y = data['bias_rating'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load RoBERTa and Tokenizer

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Tokenize data

In [20]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

In [21]:
class NewsDataset(torch.utils.data.Dataset):
    """
    We need this class to manage and load
    encoded inputs and corresponding labels into a format compatible
    with PyTorch's DataLoader for model training and evaluation :)
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)


In [ ]:
# check if a CUDA-enabled GPU is available for PyTorch
# if available, set the device to 'cuda' (GPU); otherwise, default to 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)   # move the model to the specified device (either GPU or CPU)

# Evaluate pretrained RoBERTa

In [23]:
model.eval()  # put the model in evaluation mode
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False) # initialize a DataLoader for batching during inference

predictions = [] # store predictions
actuals = []

with torch.no_grad():  # no need to track gradients during inference
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask) # forward pass
        logits = outputs.logits
        
        predicted_class = torch.argmax(logits, dim=-1) # get the predicted class

        predictions.extend(predicted_class.cpu().numpy()) # collect predictions and actual labels
        actuals.extend(labels.cpu().numpy())


In [ ]:
for i in range(5):         # first 5 predictions
    print(f"Text: {X_test[i]}")
    print(f"Predicted Class: {predictions[i]}")
    print(f"Actual Class: {y_test[i]}")
    print()

# Overall accuracy of pretrained RoBERTa on test set

In [ ]:
accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")